

> #TODO  
Hoje os IDs dos times entao sendo inseridos da forma que vem, temos que validar se aquele id de time, é o mesmo ID na tabela de **teams**

In [ ]:
pip install ipython_genutils

In [10]:
import ipynb.fs.full.FootballApi as api
import ipynb.fs.full.PostgreDB as pg
import json
import ipython_genutils

In [2]:
#paranaesne
league = 606
#season - Em caso de ligas que inician em um ano e finalizam em outro, utilizar o ano de inicio
season = 2022

In [3]:
conn = pg.PostgreDB.connect()

In [12]:
# atualizando rounds
%run ./round_updater.ipynb

185
{'get': 'fixtures/rounds', 'parameters': {'season': '2022', 'league': '606'}, 'errors': [], 'results': 11, 'paging': {'current': 1, 'total': 1}, 'response': ['Group Stage - 1', 'Group Stage - 2', 'Group Stage - 3', 'Group Stage - 4', 'Group Stage - 5', 'Group Stage - 6', 'Group Stage - 7', 'Group Stage - 8', 'Group Stage - 9', 'Group Stage - 10', 'Group Stage - 11']}


In [13]:
#importando fixture
fixtures = api.FootballApi(function=f"fixtures?league={league}&season={season}").get_data()

In [15]:
print(fixtures['response'][0])

{'fixture': {'id': 814566, 'referee': 'André Felipe Olivério', 'timezone': 'UTC', 'date': '2022-01-22T19:00:00+00:00', 'timestamp': 1642878000, 'periods': {'first': 1642878000, 'second': 1642881600}, 'venue': {'id': 230, 'name': 'Estádio Major Antônio Couto Pereira', 'city': 'Curitiba, Paraná'}, 'status': {'long': 'Match Finished', 'short': 'FT', 'elapsed': 90}}, 'league': {'id': 606, 'name': 'Paranaense - 1', 'country': 'Brazil', 'logo': 'https://media.api-sports.io/football/leagues/606.png', 'flag': 'https://media.api-sports.io/flags/br.svg', 'season': 2022, 'round': 'Group Stage - 1'}, 'teams': {'home': {'id': 147, 'name': 'Coritiba', 'logo': 'https://media.api-sports.io/football/teams/147.png', 'winner': True}, 'away': {'id': 1194, 'name': 'Cianorte', 'logo': 'https://media.api-sports.io/football/teams/1194.png', 'winner': False}}, 'goals': {'home': 1, 'away': 0}, 'score': {'halftime': {'home': 0, 'away': 0}, 'fulltime': {'home': 1, 'away': 0}, 'extratime': {'home': None, 'away': N

In [18]:
for fixture in fixtures['response']:

    fixture_id = fixture["fixture"]['id']
    fixture_referee = fixture["fixture"]['referee']
    fixture_timezone = fixture["fixture"]['timezone']
    fixture_date = fixture["fixture"]['date']
    fixture_timestamp = fixture["fixture"]['timestamp']
    
    fixture_periods = fixture["fixture"]['periods']
    fixture_first_half_start = fixture_periods['first']
    fixture_second_half_start = fixture_periods['second']

    team_home = fixture["teams"]['home']
    team_home_id = team_home['id']
    team_home_winner = team_home['winner']
    team_home_goals = fixture["goals"]['home']

    team_away = fixture["teams"]['away']
    team_away_id = team_away['id']
    team_away_winner = team_away['winner']
    team_away_goals = fixture["goals"]['away']

    team_detail_score = fixture["score"]
    team_home_ht_goals = team_detail_score["halftime"]['home']
    team_home_ft_goals = team_detail_score["fulltime"]['home']
    team_home_extratime_goals = team_detail_score["extratime"]['home']
    team_home_penalty_goals = team_detail_score["penalty"]['home']
    team_away_ht_goals = team_detail_score["halftime"]['away']
    team_away_ft_goals = team_detail_score["fulltime"]['away']
    team_away_extratime_goals = team_detail_score["extratime"]['away']
    team_away_penalty_goals = team_detail_score["penalty"]['away']

    # apenas para validacao
    league_id = fixture["league"]['id']


